In [6]:
# Milestone 1: Direct Multiple Shooting Template (CasADi)

import casadi as ca
import numpy as np

# === 1. Define the dynamic system (you can replace this with Pyridine later) ===
nx = 2  # state dimension
np_ = 1  # parameter dimension
y = ca.MX.sym("y", nx)
p = ca.MX.sym("p", np_)

# Example: simple linear system dy/dt = A*y * p
A = ca.DM([[0, 1], [-1, -0.1]])
dydt = ca.mtimes(A, y) * p  # f(y, p)
f = ca.Function("f", [y, p], [dydt])

# === 2. Set up integrator ===
dt = 0.2  # time step
integrator = ca.integrator(
    "phi", "cvodes",
    {"x": y, "p": p, "ode": dydt},
    {"tf": dt}
)

# === 3. Create shooting variables ===
m = 5  # number of shooting intervals
s_y = [ca.MX.sym(f"s_y_{i}", nx) for i in range(m + 1)]
p_sym = ca.MX.sym("p", np_)

# === 4. Define residual functions ===
F1_terms = []  # measurement residuals
F2_terms = []  # continuity constraints

# Simulate artificial measurements for testing
true_p = 2.0
true_y0 = np.array([1.0, 0.0])
tgrid = np.linspace(0, m * dt, m + 1)
ys_true = [true_y0]
for i in range(m):
    result = integrator(x0=ys_true[-1], p=true_p)
    ys_true.append(result["xf"].full().flatten())
y_meas = [y + np.random.normal(0, 0.05, size=nx) for y in ys_true]

# Build residuals and constraints
for i in range(m):
    sim = integrator(x0=s_y[i], p=p_sym)
    y_end = sim["xf"]
    F2_terms.append(s_y[i+1] - y_end)  # continuity
    F1_terms.append(s_y[i+1] - y_meas[i+1])  # measurement residual (simple version)

F1 = ca.vertcat(*F1_terms)
F2 = ca.vertcat(*F2_terms)
z = ca.vertcat(*s_y, p_sym)

F1_fun = ca.Function("F1", [z], [F1])
F2_fun = ca.Function("F2", [z], [F2])

print("Direct Multiple Shooting problem successfully built.")


Direct Multiple Shooting problem successfully built.
